# 1. Install and Import Dependencies

In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.2 MB/s eta 0:00:00


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [61]:
tokens = tokenizer.encode('This is an amazing place, love it', return_tensors='pt')

In [62]:
tokens

tensor([[  101, 10372, 10127, 10144, 39854, 11125,   117, 11157, 10197,   102]])

In [63]:
#tokenizer.decode(tokens[0])

In [64]:
result = model(tokens)

In [65]:
result.logits

tensor([[-2.3497, -2.7211, -1.0434,  1.4095,  3.7971]],
       grad_fn=<AddmmBackward0>)

In [66]:
int(torch.argmax(result.logits))+1

5

# 4. Collect Reviews

In [39]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [40]:
reviews

["Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!",
 "Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. We all enjoyed our food, as well as our coffee and tea drinks.We were greeted immediately by a friendly server asking if we would like to sit inside or out. We said we would like inside, but weren't exactly sure how many were joining us yet- at least 4. 

# 5. Load Reviews into DataFrame and Score

In [42]:
import numpy as np
import pandas as pd

In [44]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [46]:
df.head()

,review
0,Very cute coffee shop and restaurant. They hav...
1,Six of us met here for breakfast before our wa...
2,Great place with delicious food and friendly s...
3,Some of the best Milkshakes me and my daughter...
4,Great food amazing coffee and tea. Short walk ...


In [50]:
df.shape

(10, 1)

In [45]:
df['review'].iloc[0]

"Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!"

In [47]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [48]:
sentiment_score(df['review'].iloc[1])

4

In [51]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

512 to grab everything before 512 token value in each review, this nlp pipeline is limited to how much tokens you can pass to the model at one time

In [52]:
df

,review,sentiment
0,Very cute coffee shop and restaurant. They hav...,4
1,Six of us met here for breakfast before our wa...,4
2,Great place with delicious food and friendly s...,5
3,Some of the best Milkshakes me and my daughter...,5
4,Great food amazing coffee and tea. Short walk ...,5
5,It was ok. Had coffee with my friends. I'm new...,3
6,Ricotta hot cakes! These were so yummy. I ate ...,5
7,We came for brunch twice in our week-long visi...,4
8,Great staff and food. Must try is the pan fri...,5
9,I came to Social brew cafe for brunch while ex...,5


In [55]:
print(df['review'].iloc[3])
print(df['sentiment'].iloc[3])

Some of the best Milkshakes me and my daughter ever tasted. MMMMMM HMMMMMMMM.
5


In [60]:
def specific_rev(line):
  print(df['review'].iloc[line])
  print(df['sentiment'].iloc[line])

specific_rev(7)

We came for brunch twice in our week-long visit to Sydney. Everything on the menu not only sounds delicious, but is really tasty. It really gave us a sour taste of how bad breaky is in America with what's so readily available in Sydney!  Both days we went were Saturdays and there was a bit of a wait to be seated, the cafe is extremely busy for both dine-in and take-away. Service is fairly quick and servers are all friendly. The location is in Surrey Hills a couple blocks away from the bustling touristy Darling Harbor.The green smoothie is very tasty and refreshing. We tried the smoked salmon salad, the soft shell crab tacos, ricotta hotcakes, and the breaky sandwich. All were delicious, well seasoned, and a solid amount of food for the price. A definite recommend for anyone's trip into Sydney!
4
